In [98]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import re

In [99]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [100]:
def bookGrab(x):
        title = book.h3.a["title"]
        x.append(title)



In [101]:
# Scrape All Books Basic info
url = 'https://books.toscrape.com/'
browser.visit(url)
titles = []
prices = []
ratings = []
stocks = []
for x in range(1, 4):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    books = soup.find_all('article', class_='product_pod')
    for book in books:
        title = book.h3.a["title"]
        price = book.find("p", class_="price_color").get_text().strip()
        rating = book.p["class"][1]
        stock = book.find("p", class_="instock availability").get_text().strip()
        titles.append(title)
        prices.append(price)
        ratings.append(rating)
        stocks.append(stock)
    if browser.links.find_by_partial_text('next'):
        browser.links.find_by_partial_text('next').click()
    else:
        print("-Done")

In [102]:
# Crate Basic Scrape Data Frame
scrape_df = pd.DataFrame(
                        {'Titles':  titles,
                        'Prices':   prices,
                        'Ratings':  ratings,
                        'Stocks':   stocks})
scrape_df

,Titles,Prices,Ratings,Stocks
0,A Light in the Attic,£51.77,Three,In stock
1,Tipping the Velvet,£53.74,One,In stock
2,Soumission,£50.10,One,In stock
3,Sharp Objects,£47.82,Four,In stock
4,Sapiens: A Brief History of Humankind,£54.23,Five,In stock
5,The Requiem Red,£22.65,One,In stock
6,The Dirty Little Secrets of Getting Your Dream...,£33.34,Four,In stock
7,The Coming Woman: A Novel Based on the Life of...,£17.93,Three,In stock
8,The Boys in the Boat: Nine Americans and Their...,£22.60,Four,In stock
9,The Black Maria,£52.15,One,In stock


In [103]:
# Collect Category links
url = 'https://books.toscrape.com/'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

Section = soup.find('ul', class_='nav nav-list')
linksSection = Section.find_all('a')
linksSectionStr = []
for x in linksSection:
    linksSectionStr.append(str(x))
categoriesLinks = []
for x in range(len(linksSectionStr)):
    linksSectionStr[x] = re.sub(' +', '', linksSectionStr[x])
    linksSectionStr[x] = re.sub('\n', '', linksSectionStr[x])
    linksSectionStr[x] = re.findall(r'"([^"]*)"', linksSectionStr[x])
    categoriesLinks.append(linksSectionStr[x][0])
print(categoriesLinks)


['catalogue/category/books_1/index.html', 'catalogue/category/books/travel_2/index.html', 'catalogue/category/books/mystery_3/index.html', 'catalogue/category/books/historical-fiction_4/index.html', 'catalogue/category/books/sequential-art_5/index.html', 'catalogue/category/books/classics_6/index.html', 'catalogue/category/books/philosophy_7/index.html', 'catalogue/category/books/romance_8/index.html', 'catalogue/category/books/womens-fiction_9/index.html', 'catalogue/category/books/fiction_10/index.html', 'catalogue/category/books/childrens_11/index.html', 'catalogue/category/books/religion_12/index.html', 'catalogue/category/books/nonfiction_13/index.html', 'catalogue/category/books/music_14/index.html', 'catalogue/category/books/default_15/index.html', 'catalogue/category/books/science-fiction_16/index.html', 'catalogue/category/books/sports-and-games_17/index.html', 'catalogue/category/books/add-a-comment_18/index.html', 'catalogue/category/books/fantasy_19/index.html', 'catalogue/

In [104]:
temListOfCategories = ["Travel","Mystery","Historical_Fiction","Sequential_Art","Classics","Philosophy","Romance","Womens_Fiction","Fiction","Childrens","Religion","Nonfiction","Music","Default","Science_Fiction","Sports_and_Games","Add_a_comment","Fantasy","New_Adult","Young_Adult","Science","Poetry","Paranormal","Art","Psychology","Autobiography","Parenting","Adult_Fiction","Humor","Horror","History","Food_and_Drink","Christian_Fiction","Business","Biography","Thriller","Contemporary","Spirituality","Academic","Self_Help","Historical","Christian","Suspense","Short_Stories","Novels","Health","Politics","Cultural","Erotica","Crime"]

In [105]:
url = 'https://books.toscrape.com/'

generaCollection = []
genraTitles = []
for x in range(1,5):
    browser.visit(url + categoriesLinks[x])
    title1 = []
    page = True
    while page == True:
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        books = soup.find_all('article', class_='product_pod')
        for book in books:
            title = book.h3.a["title"]
            title1.append(title)
        if browser.links.find_by_partial_text('next'):
            browser.links.find_by_partial_text('next').click()
        else:
            print("-Done")
            page = False
    generaCollection.append(title1)



-Done
-Done
-Done
-Done


In [110]:
len(generaCollection)

4

In [123]:
toAdd = pd.DataFrame()
ahh = pd.DataFrame()
woo = True
for x in range(0,4):
    while woo == True:
        ahh = ahh.append(scrape_df[scrape_df['Titles'].isin(generaCollection[x])])
        ahh['Genre'] = x
        woo = False
    woo = True
    toAdd = toAdd.append(ahh)
    ahh.drop(ahh.index, inplace=True)
toAdd


C:\Users\pereg\AppData\Local\Temp\ipykernel_18520\3241307790.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ahh = ahh.append(scrape_df[scrape_df['Titles'].isin(generaCollection[x])])
C:\Users\pereg\AppData\Local\Temp\ipykernel_18520\3241307790.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  toAdd = toAdd.append(ahh)
C:\Users\pereg\AppData\Local\Temp\ipykernel_18520\3241307790.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ahh = ahh.append(scrape_df[scrape_df['Titles'].isin(generaCollection[x])])
C:\Users\pereg\AppData\Local\Temp\ipykernel_18520\3241307790.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  toAdd = toAdd.

,Titles,Prices,Ratings,Stocks,Genre
19,It's Only the Himalayas,£45.17,Two,In stock,0
3,Sharp Objects,£47.82,Four,In stock,1
37,"In a Dark, Dark Wood",£19.63,One,In stock,1
58,The Past Never Ends,£56.50,Four,In stock,1
1,Tipping the Velvet,£53.74,One,In stock,2
13,Scott Pilgrim's Precious Little Life (Scott Pi...,£52.29,Five,In stock,3
51,Tsubasa: WoRLD CHRoNiCLE 2 (Tsubasa WoRLD CHRo...,£16.28,One,In stock,3
53,This One Summer,£19.49,Four,In stock,3


In [114]:
toAdd = toAdd.drop(["Prices","Ratings","Stocks"], axis=1)
# scrape_df.where(scrape_df == "It's Only the Himalayas")
df = scrape_df.merge(toAdd, how='outer',on='Titles')
# df=scrape_df.join(toAdd, lsuffix="_left", rsuffix="_right")



KeyError: "['Prices', 'Ratings', 'Stocks'] not found in axis"

In [108]:
df

,Titles,Prices,Ratings,Stocks,Genre
0,A Light in the Attic,£51.77,Three,In stock,NaN
1,Tipping the Velvet,£53.74,One,In stock,NaN
2,Soumission,£50.10,One,In stock,NaN
3,Sharp Objects,£47.82,Four,In stock,NaN
4,Sapiens: A Brief History of Humankind,£54.23,Five,In stock,NaN
5,The Requiem Red,£22.65,One,In stock,NaN
6,The Dirty Little Secrets of Getting Your Dream...,£33.34,Four,In stock,NaN
7,The Coming Woman: A Novel Based on the Life of...,£17.93,Three,In stock,NaN
8,The Boys in the Boat: Nine Americans and Their...,£22.60,Four,In stock,NaN
9,The Black Maria,£52.15,One,In stock,NaN


In [109]:
browser.quit()